In [1]:
import pandas as pd

from src.path import DataPaths
from src.merge import merge_dataframe_rows

from tools.type_check import print_detailed_info
from tools.save_data import export_dataframe



df = pd.read_parquet(DataPaths.test_parquet)
print_detailed_info(df)

Original dataset: 21,946 rows
Number of columns in the DataFrame: 28
Data columns (total 28 columns):
 #   Column                               Actual type     Preview
---  ------------------------------------ ---------------- --------------------------------------------------
 0   unspsc                               str             Sewing and stitchery and weaving equipme...
 1   root_domain                          str             studio-atcoat.com
 2   page_url                             str             https://studio-atcoat.com/1372696759/?id...
 3   product_title                        str             Glimakra Warping Board (8m)
 4   product_identifier                   ndarray(0,)     []
 5   brand                                str             CST
 6   intended_industries                  ndarray(1,)     ['Textile']
 7   applicability                        ndarray(1,)     ['use with floor looms']
 8   eco_friendly                         bool            True
 9   ethical_and_

In [2]:
df.head(10)

,unspsc,root_domain,page_url,product_title,product_identifier,brand,intended_industries,applicability,eco_friendly,ethical_and_sustainability_practices,...,form,size,color,purity,energy_efficiency,pressure_rating,power_rating,quality_standards_and_certifications,miscellaneous_features,merged_description
0,Sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,Glimakra Warping Board (8m),[],None,[Textile],[use with floor looms],None,[],...,[],"[{'dimension': 'Length', 'qualitative': False,...",[],[],[],[],[],[],[],The Glimakra Warping Board is designed for use...
1,Electric alternating current AC motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,NMRV Worm Gearbox Motor,[],None,[Industrial],[industrial applications],None,[],...,[],[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...",[],[],[],"[{'qualitative': False, 'type': 'min', 'unit':...",[],"[Omnibearing installation, High radiation effi...",The NMRV Worm Gearbox Motor is a high-efficien...
2,Vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,Nissan R33 GTR Car Cover,[],None,[Automotive],[protecting vehicles from the elements],None,[],...,[],[],[],[],[],[],[],[],"[Personalization with custom brand logos, grap...","The ""Car Cover"" is a custom-designed cover tai..."
3,Pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,Flexible Fittings,[],None,[Plumbing],[plumbing installations],None,[],...,[],[],[],[],[],[],[],[],"[allows for movement, flexibility in installat...","""Flexible Fittings"" are designed for plumbing ..."
4,Doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,CST-HGD-33103 Hinged Closet Door,[],CST,"[Home Appliances, Construction]",[Closet Storage],None,[],...,[],[],[],[],[],[],[],[],"[Italian craftsmanship, German engineering, Sm...",The CST-HGD-33103 Hinged Closet Door is a meti...
5,Faucets or taps,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,Deep Faucets,[],None,[Plumbing],[],None,[],...,[],[],[],[],[],[],[],[],"[deep design, secure and stable connection]","""Deep Faucets"" are designed with a deep design..."
6,Dispensing tools,advancedpressuresystems.ca,https://advancedpressuresystems.ca/collections...,10K Dry Shut-Off Gun Handle Assembly,[],None,[Manufacturing],[High-Pressure Water Blasting Operations],None,[],...,[],[],[],[],[],"[{'qualitative': True, 'type': 'exact', 'unit'...",[],[],[],The 10K Dry Shut-Off Gun Handle Assembly is a ...
7,Medical facility materials handling and distri...,armstrongmedical.com,https://www.armstrongmedical.com//cart-systems...,Cranberry Cart Systems,[],Armstrong Medical,[Healthcare],[medical and healthcare applications],None,[],...,[],[],"[{'original': 'Cranberry', 'simple': 'Red'}]",[],[],[],[],[],[vibrant and eye-catching look],"""Cranberry Cart Systems"" from the Armstrong Me..."
8,Pneumatic tools,advancedpressuresystems.ca,https://advancedpressuresystems.ca/collections...,10K Air Operated Control Gun,[],None,[Manufacturing],[water blasting operations],None,[],...,[],"[{'dimension': None, 'qualitative': True, 'typ...",[],[],[],[],[],[],[],"The ""Air Operated Control Gun"" is designed for..."
9,Tshirts,workwonderly.com,https://www.workwonderly.com/tags/Medicine/col...,5 THINGS YOU SHOULD KNOW ABOUT MY NURSE PRACTI...,[],None,"[Fashion, Retail]",[Wearing],None,[],...,[],[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...",[],[],[],[],[],[Long sleeve],"The ""Long Sleeve Tee"" is a long sleeve t-shirt..."


In [3]:
# Step 1 Merging by common key, page_url and product_title
df['key'] = df['page_url'] + '|' + df['product_title']

# Print initial number of rows
initial_rows = len(df)
print(f"Initial number of rows: {initial_rows}")

# Find the duplicate rows based on the key column
duplicated_mask = df.duplicated(subset=['key'], keep=False)
duplicates_df = df[duplicated_mask].copy()
duplicate_count = len(duplicates_df)

export_dataframe(duplicates_df, DataPaths.visualization_merge_url_title_dir, 'duplicates_before_merge', file_format='csv')

merged_df = merge_dataframe_rows(df, key_column='key')

# Before dropping the key column, identify the duplicates
merged_duplicates = merged_df[merged_df['key'].isin(duplicates_df['key'].unique())].copy()

# Calculate how many rows were removed through merging
merged_rows = len(merged_df)
rows_removed = initial_rows - merged_rows
percent_removed = rows_removed / initial_rows

print(f"Rows after merging: {merged_rows}")
print(f"Rows removed through merging: {rows_removed} ({percent_removed:.2%} of total)")

# Now it's safe to drop the key column from both dataframes
merged_df = merged_df.drop(columns=['key'])
merged_duplicates = merged_duplicates.drop(columns=['key'])

print(f"Number of previously duplicate rows after merging: {len(merged_duplicates)}")

export_dataframe(merged_duplicates, DataPaths.visualization_merge_url_title_dir, 'duplicates_after_merge', file_format='csv')
export_dataframe(merged_df, DataPaths.visualization_merge_url_title_dir, 'merged_url_title', file_format='csv')
export_dataframe(merged_df, DataPaths.parquet_merge_url_title_dir, 'merged_url_title', file_format='parquet')

Initial number of rows: 21946
Exported data to: E:\veridion_deduplication\data\visualization\processed\2_merge_url_title\duplicates_before_merge.csv
Rows after merging: 21937
Rows removed through merging: 9 (0.04% of total)
Number of previously duplicate rows after merging: 9
Exported data to: E:\veridion_deduplication\data\visualization\processed\2_merge_url_title\duplicates_after_merge.csv
Exported data to: E:\veridion_deduplication\data\visualization\processed\2_merge_url_title\merged_url_title.csv
Exported data to: E:\veridion_deduplication\data\parquet\processed\2_merge_url_title\merged_url_title.snappy.parquet


WindowsPath('E:/veridion_deduplication/data/parquet/processed/2_merge_url_title/merged_url_title.snappy.parquet')

In [4]:
# Step 2 Merging by common key, product_title and root_domain
print("\n" + "="*50)
print("STEP 2: Merging by product_title and root_domain")
print("="*50)

# Create a new key column by combining product_title and root_domain
df['key'] = df['product_title'] + '|' + df['root_domain']

# Print initial number of rows
initial_rows = len(df)
print(f"Initial number of rows: {initial_rows}")

# Find the duplicate rows based on the new key column
duplicated_mask = df.duplicated(subset=['key'], keep=False)
duplicates_df = df[duplicated_mask].copy()
duplicate_count = len(duplicates_df)
print(f"Number of duplicate rows identified: {duplicate_count} ({duplicate_count/initial_rows:.2%} of total)")

export_dataframe(duplicates_df, DataPaths.visualization_merge_title_domain_dir, 'duplicates_before_merge', file_format='csv')

# Merge the dataframe rows based on the new key
merged_df = merge_dataframe_rows(df, key_column='key')

# Before dropping the key column, identify the duplicates
merged_duplicates = merged_df[merged_df['key'].isin(duplicates_df['key'].unique())].copy()

# Calculate how many rows were removed through merging
merged_rows = len(merged_df)
rows_removed = initial_rows - merged_rows
percent_removed = rows_removed / initial_rows

print(f"Rows after merging: {merged_rows}")
print(f"Rows removed through merging: {rows_removed} ({percent_removed:.2%} of total)")

# Now it's safe to drop the key column from both dataframes
merged_df = merged_df.drop(columns=['key'])
merged_duplicates = merged_duplicates.drop(columns=['key'])

print(f"Number of previously duplicate rows after merging: {len(merged_duplicates)}")

export_dataframe(merged_duplicates, DataPaths.visualization_merge_title_domain_dir, 'duplicates_after_merge', file_format='csv')
export_dataframe(merged_df, DataPaths.visualization_merge_title_domain_dir, 'merged_title_domain', file_format='csv')
export_dataframe(merged_df, DataPaths.parquet_merge_title_domain_dir, 'merged_title_domain', file_format='parquet')

export_dataframe(merged_df, DataPaths.visualization_final_dir, 'final_data', file_format='csv')
export_dataframe(merged_df, DataPaths.parquet_final_dir, 'final_data', file_format='parquet')


STEP 2: Merging by product_title and root_domain
Initial number of rows: 21946
Number of duplicate rows identified: 4257 (19.40% of total)
Exported data to: E:\veridion_deduplication\data\visualization\processed\3_merge_title_domain\duplicates_before_merge.csv
Logged 2 rows with merge errors to E:\veridion_deduplication\data\error\merge_errors.csv
Rows after merging: 19054
Rows removed through merging: 2892 (13.18% of total)
Number of previously duplicate rows after merging: 1365
Exported data to: E:\veridion_deduplication\data\visualization\processed\3_merge_title_domain\duplicates_after_merge.csv
Exported data to: E:\veridion_deduplication\data\visualization\processed\3_merge_title_domain\merged_title_domain.csv
Exported data to: E:\veridion_deduplication\data\parquet\processed\3_merge_title_domain\merged_title_domain.snappy.parquet
Exported data to: E:\veridion_deduplication\data\visualization\final\final_data.csv
Exported data to: E:\veridion_deduplication\data\parquet\final\final

WindowsPath('E:/veridion_deduplication/data/parquet/final/final_data.snappy.parquet')

In [5]:
df_original_data = pd.read_parquet(DataPaths.file_parquet_original)
df_final_data = pd.read_parquet(DataPaths.file_parquet_final)

In [6]:
import os
import numpy as np

# Get file paths from your existing DataPaths class
original_path = DataPaths.file_parquet_original
final_path = DataPaths.file_parquet_final

# Get file sizes
original_size_bytes = os.path.getsize(original_path)
final_size_bytes = os.path.getsize(final_path)

original_size_mb = original_size_bytes / (1024 * 1024)
final_size_mb = final_size_bytes / (1024 * 1024)

# Calculate differences
row_diff = len(df_final_data) - len(df_original_data)
row_diff_pct = (row_diff / len(df_original_data)) * 100 if len(df_original_data) > 0 else np.inf

col_diff = len(df_final_data.columns) - len(df_original_data.columns)
col_diff_pct = (col_diff / len(df_original_data.columns)) * 100 if len(df_original_data.columns) > 0 else np.inf

size_diff = final_size_mb - original_size_mb
size_diff_pct = (size_diff / original_size_mb) * 100 if original_size_mb > 0 else np.inf

# Find added/removed columns
original_cols = set(df_original_data.columns)
final_cols = set(df_final_data.columns)

added_cols = final_cols - original_cols
removed_cols = original_cols - final_cols

# Format output
print("\n" + "="*50)
print("DATAFRAME COMPARISON ANALYSIS")
print("="*50)

print("\nROW COMPARISON:")
print(f"Original rows: {len(df_original_data):,}")
print(f"Final rows: {len(df_final_data):,}")
print(f"Difference: {row_diff:,} rows ({row_diff_pct:.2f}%)")

print("\nCOLUMN COMPARISON:")
print(f"Original columns: {len(df_original_data.columns)}")
print(f"Final columns: {len(df_final_data.columns)}")
print(f"Difference: {col_diff} columns ({col_diff_pct:.2f}%)")

if added_cols:
    print(f"\nAdded columns ({len(added_cols)}):")
    for col in sorted(added_cols):
        print(f"  - {col}")

if removed_cols:
    print(f"\nRemoved columns ({len(removed_cols)}):")
    for col in sorted(removed_cols):
        print(f"  - {col}")

print("\nFILE SIZE COMPARISON:")
print(f"Original size: {original_size_mb:.2f} MB")
print(f"Final size: {final_size_mb:.2f} MB")
print(f"Difference: {size_diff:.2f} MB ({size_diff_pct:.2f}%)")

# Generate summary table
print("\nSUMMARY TABLE:")
print(f"{'Metric':<15} {'Original':<15} {'Final':<15} {'Difference':<15} {'Percentage':<15}")
print("-"*75)
print(f"{'Rows':<15} {len(df_original_data):<15,} {len(df_final_data):<15,} {row_diff:<15,} {row_diff_pct:<15.2f}%")
print(f"{'Columns':<15} {len(df_original_data.columns):<15} {len(df_final_data.columns):<15} {col_diff:<15} {col_diff_pct:<15.2f}%")
print(f"{'Size (MB)':<15} {original_size_mb:<15.2f} {final_size_mb:<15.2f} {size_diff:<15.2f} {size_diff_pct:<15.2f}%")


DATAFRAME COMPARISON ANALYSIS

ROW COMPARISON:
Original rows: 21,946
Final rows: 19,054
Difference: -2,892 rows (-13.18%)

COLUMN COMPARISON:
Original columns: 31
Final columns: 28
Difference: -3 columns (-9.68%)

Added columns (1):
  - merged_description

Removed columns (4):
  - description
  - manufacturing_year
  - product_name
  - product_summary

FILE SIZE COMPARISON:
Original size: 10.72 MB
Final size: 6.79 MB
Difference: -3.93 MB (-36.70%)

SUMMARY TABLE:
Metric          Original        Final           Difference      Percentage     
---------------------------------------------------------------------------
Rows            21,946          19,054          -2,892          -13.18         %
Columns         31              28              -3              -9.68          %
Size (MB)       10.72           6.79            -3.93           -36.70         %
